In [1]:
from pyspark.sql.functions import sum, col, concat
from pyspark.sql.functions import to_timestamp, date_format
from pyspark.sql.functions import desc, last_day, rand, lpad

StatementMeta(, ed784977-e7be-478d-b2c2-5ea284d7cebb, 3, Finished, Available)

## **Import Dim Tables**

In [2]:
dim_customer = spark.sql("SELECT * FROM LTT_GoldLakehouse.dim_customer")
dim_product = spark.sql("SELECT * FROM LTT_GoldLakehouse.dim_product")
dim_category = spark.sql("SELECT * FROM LTT_GoldLakehouse.dim_category")
dim_department = spark.sql("SELECT * FROM LTT_GoldLakehouse.dim_department")

dim_delivery_status = spark.sql("SELECT * FROM LTT_GoldLakehouse.dim_delivery_status")
dim_delivery_risk = spark.sql("SELECT * FROM LTT_GoldLakehouse.dim_delivery_risk")
dim_shipping = spark.sql("SELECT * FROM LTT_GoldLakehouse.dim_shipping_mode")
dim_transaction = spark.sql("SELECT * FROM LTT_GoldLakehouse.dim_transaction_type")
dim_order_status = spark.sql("SELECT * FROM LTT_GoldLakehouse.dim_order_status")

dim_date = spark.sql("SELECT * FROM LTT_GoldLakehouse.dim_date")
dim_time = spark.sql("SELECT * FROM LTT_GoldLakehouse.dim_time")

dim_store = spark.sql("SELECT * FROM LTT_GoldLakehouse.dim_store")
dim_destination = spark.sql("SELECT * FROM LTT_GoldLakehouse.dim_destination")

fact_sales = spark.sql("SELECT * FROM LTT_GoldLakehouse.fact_sales")

StatementMeta(, ed784977-e7be-478d-b2c2-5ea284d7cebb, 4, Finished, Available)

In [3]:
fact_sales.count()

StatementMeta(, ed784977-e7be-478d-b2c2-5ea284d7cebb, 5, Finished, Available)

100

## **Merge Queries**

In [4]:
fact_sales.createOrReplaceTempView("fact_sales")
display(fact_sales.head(5))

StatementMeta(, ed784977-e7be-478d-b2c2-5ea284d7cebb, 6, Finished, Available)

SynapseWidget(Synapse.DataFrame, a3a7a6e7-59f7-4019-9b1b-06b39c04afac)

## Create monthly facts for warehouse

> PHÂN TÍCH DOANH SỐ BÁN HÀNG CỦA TỪNG BƯU CỤC THEO THÁNG
* Tính doanh số bán hàng, lợi nhuận hàng tháng trong thành phố của công ty khi nhận đơn đặt hàng từ các cửa hàng
* Đếm số lượng đơn hàng hàng tháng của cửa hàng đó

## Create monthly facts

> PHÂN TÍCH DOANH SỐ BÁN HÀNG CỦA TỪNG CỬA HÀNG THEO THÁNG
* Tính doanh số bán hàng, lợi nhuận hàng tháng của công ty khi nhận đơn đặt hàng từ các cửa hàng
* Đếm số lượng đơn hàng hàng tháng của cửa hàng đó

In [5]:
# Doanh so ban hang va loi nhuan cua tung cua hang theo thang nam
# Tong so luong don dat hang moi thang cua tung cua hang
store_monthly_sale = (spark.sql("""
    with 
    store_monthly_sale as (
        select 
            ss.store_key,
            dd.year,
            dd.month,
            count(ss.order_id) as monthly_total_order,
            sum(sale_amount) as monthly_sale_amount,
            sum(benefit_amount) as monthly_benefit_amount
        from (
            select
                order_id,
                order_date_key,
                store_key,
                sum(sales) as sale_amount,
                sum(benefit_per_order) as benefit_amount
            from fact_sales
            group by
                order_id,
                order_date_key,
                store_key
        ) as ss -- store sale
        join dim_date as dd -- get month
        on ss.order_date_key = dd.date_key
        group by 
            dd.year,
            dd.month,
            ss.store_key
    )
    select *
    from store_monthly_sale
""").orderBy('store_key'))

StatementMeta(, ed784977-e7be-478d-b2c2-5ea284d7cebb, 7, Finished, Available)

In [6]:
store_monthly_sale.createOrReplaceTempView('store_monthly_sale')
store_monthly_sale.show(5)

StatementMeta(, ed784977-e7be-478d-b2c2-5ea284d7cebb, 8, Finished, Available)

+---------+----+-----+-------------------+-------------------+----------------------+
|store_key|year|month|monthly_total_order|monthly_sale_amount|monthly_benefit_amount|
+---------+----+-----+-------------------+-------------------+----------------------+
|        1|2015|    1|                  1|        224.9499969|           21.10000038|
|        2|2015|    1|                  1|        49.97999954|                  5.25|
|        3|2015|    1|                  1|               75.0|          -5.510000229|
|        4|2015|    1|                  1|        299.9700012|           100.3799973|
|        5|2015|    1|                  1|        299.9500122|           34.11999893|
+---------+----+-----+-------------------+-------------------+----------------------+
only showing top 5 rows



In [7]:
store_monthly_sales = store_monthly_sale.withColumn('date_key', 
    concat(
        col('year'), 
        lpad(col('month'), 2, "0"), 
        lpad(date_format(last_day(concat(col('year'), col('month'))), 'dd'), 2, "0")
    )
)

StatementMeta(, ed784977-e7be-478d-b2c2-5ea284d7cebb, 9, Finished, Available)

In [8]:
store_monthly_sales.count()

StatementMeta(, ed784977-e7be-478d-b2c2-5ea284d7cebb, 10, Finished, Available)

72

In [9]:
display(store_monthly_sales.head(5))

StatementMeta(, ed784977-e7be-478d-b2c2-5ea284d7cebb, 11, Finished, Available)

SynapseWidget(Synapse.DataFrame, a74a297f-08b0-4f22-ad0d-14c21e2ad585)

In [10]:
lower_bound = 0.3
upper_bound = 1.7
# Apply the random scaling to sales, total orders, and benefit
store_monthly_sales = store_monthly_sales \
    .withColumn('sales_plan', col('monthly_sale_amount') * (lower_bound + rand() * (upper_bound - lower_bound))) \
    .withColumn('total_orders_plan', col('monthly_total_order') * (lower_bound + rand() * (upper_bound - lower_bound))) \
    .withColumn('benefit_plan', col('monthly_benefit_amount') * (lower_bound + rand() * (upper_bound - lower_bound))) \
    .select('store_key', 'date_key', 'monthly_sale_amount', 'sales_plan', 'monthly_total_order', 'total_orders_plan', 'monthly_benefit_amount', 'benefit_plan')

# Show the modified DataFrame
display(store_monthly_sales.head(5))


StatementMeta(, ed784977-e7be-478d-b2c2-5ea284d7cebb, 12, Finished, Available)

SynapseWidget(Synapse.DataFrame, 3d2cb98e-c185-4f9a-9e49-f2828ab34625)

> Warehouse

In [11]:
display(
spark.sql("""
select order_id, order_date_key, 
    store_country as country, 
    store_state as state,
    store_city as warehouse_name, 
    sum(ss.sale_amount) as warehouse_sale,
    sum(ss.benefit_amount) as warehouse_benefit,
    avg(latitude) as latitude, avg(longitude) as longitude
from (
    select
        order_id,
        order_date_key,
        store_key,
        sum(sales) as sale_amount,
        sum(benefit_per_order) as benefit_amount
    from fact_sales
    group by
        order_id,
        order_date_key,
        store_key
) as ss -- store sale
join dim_store ds
on ss.store_key = ds.store_key
group by order_id, order_date_key, store_city, store_country, store_state
""").head(5))

StatementMeta(, ed784977-e7be-478d-b2c2-5ea284d7cebb, 13, Finished, Available)

SynapseWidget(Synapse.DataFrame, f98b95b5-4dfd-4fb8-9c9e-33ca785e8290)

> PHÂN TÍCH DOANH SỐ BÁN HÀNG CỦA TỪNG CỬA HÀNG THEO TỪNG KHUNG GIỜ (BLOCKS)
* Đếm số lượng bán đơn hàng bán ra của từng khung giờ

> PHÂN TÍCH DOANH SỐ BÁN HÀNG CỦA TỪNG NGÀNH HÀNG THEO THÁNG
* Đếm số lượng đơn hàng bán ra của ngành hàng đó
* Tính tổng doanh số bán ra, tổng lợi nhuận của từng ngành hàng


## Load into GoldLakehouse

In [12]:
store_monthly_sales.write.format('delta').mode('overwrite').saveAsTable('LTT_GoldLakehouse.fact_monthly_store_sales')

StatementMeta(, ed784977-e7be-478d-b2c2-5ea284d7cebb, 14, Finished, Available)